In [1]:
import numpy as np
import torch as th


from glob import glob
from utils import *
import ray
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ray.init(num_cpus=160, ignore_reinit_error=True)

2023-08-25 10:58:50,373	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.11.3
Ray version:,2.6.2
Dashboard:,http://127.0.0.1:8266


In [3]:
insmp = Backtester(20190103, 20210101)
otsmp = Backtester(20210101, 20211231)

num of factors:124
num of factors:124


In [4]:
factors = insmp.calc_factor()

calc_factor took 239 seconds


In [5]:
factors2 = otsmp.calc_factor()

calc_factor took 140 seconds


In [7]:
df_ic = insmp.calc_ic()
df_ic2 = otsmp.calc_ic()

calc_ic took 584 seconds
calc_ic took 285 seconds


In [8]:
insmp.df_ic.query("y1>.01 and y2>.015 and y5>.02")

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.042246,0.046690,0.060397
20230813143935_satd_lexpr08_lopt34_10_0322,0.019739,0.023186,0.032060
20230813143936_satd_lexpr08_lopt34_10_0714,0.020054,0.023814,0.034957
20230813145451_satd_lexpr08_lopt34_10_0856,0.045665,0.051074,0.071974
20230813145458_satd_lexpr08_lopt34_10_0262,0.039762,0.043808,0.061600
...,...,...,...
20230825003101_satd_lexpr10_lopt50_10_3581,0.048912,0.053101,0.063885
20230825090920_satd_lexpr10_lopt50_10_9548,0.014690,0.017996,0.025416
20230825090924_satd_lexpr10_lopt50_10_9750,0.013659,0.015380,0.028290
20230825090927_satd_lexpr10_lopt50_10_5052,0.028705,0.031422,0.040526


In [9]:
otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02")

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.029942,0.032678,0.039297
20230813143935_satd_lexpr08_lopt34_10_0322,0.011797,0.013961,0.012473
20230813145451_satd_lexpr08_lopt34_10_0856,0.026510,0.029783,0.036930
20230813145458_satd_lexpr08_lopt34_10_0262,0.023295,0.025203,0.033990
20230813145501_satd_lexpr08_lopt34_10_0726,0.029467,0.029454,0.030696
...,...,...,...
20230825003035_satd_lexpr10_lopt50_10_6173,0.019370,0.022707,0.023508
20230825003047_satd_lexpr10_lopt50_10_5577,0.025244,0.027922,0.030913
20230825003101_satd_lexpr10_lopt50_10_3581,0.026536,0.029207,0.036954
20230825090927_satd_lexpr10_lopt50_10_5052,0.015517,0.017328,0.017932


In [10]:
selected_idx = otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02").index

In [11]:
feature_dict = dict(zip(otsmp.factor_names,otsmp.factors))

In [15]:
selected_sigs = [feature_dict[i] for i in selected_idx]

In [17]:
def calc_sig_corr(sigs):
    n = len(sigs)
    df_corr = np.ones((n,n))
    corrs = []
    for i in range(n):
        for j in range(i+1,n):
            ic = torch.nanmean(batch_pearsonr(sigs[i],sigs[j],))
            df_corr[i,j] = ic
            df_corr[j,i] = ic
    return df_corr
            

In [19]:
df_corr = calc_sig_corr(selected_sigs)

In [21]:
df_corr.shape

(87, 87)

In [23]:
df_corr = pd.DataFrame(df_corr)

In [26]:
sigs = remove_high_corr_factors(df_corr.values)

In [28]:
len(sigs)

64

In [36]:
final = [selected_idx[idx] for idx in sigs]

In [47]:

# 设置显示最大列数
pd.set_option('display.max_columns', None)

# 设置显示最大行数
pd.set_option('display.max_rows', None)
finaldf =  otsmp.df_ic.loc[final]

In [49]:
print(finaldf)

                                                  y1        y2        y5
20230811171800_satd_lexpr08_lopt34_10_0042  0.029942  0.032678  0.039297
20230813143935_satd_lexpr08_lopt34_10_0322  0.011797  0.013961  0.012473
20230813145451_satd_lexpr08_lopt34_10_0856  0.026510  0.029783  0.036930
20230813145501_satd_lexpr08_lopt34_10_0726  0.029467  0.029454  0.030696
20230814092613_satd_lexpr08_lopt34_10_0788  0.030076  0.031085  0.033477
20230814092614_satd_lexpr08_lopt34_10_0177  0.010392  0.011457  0.016512
20230814092615_satd_lexpr08_lopt34_10_0018  0.022174  0.023574  0.024581
20230814092615_satd_lexpr08_lopt34_10_0143  0.022328  0.025780  0.027584
20230814095857_satd_lexpr08_lopt34_10_0578  0.028826  0.031991  0.031900
20230814102408_satd_lexpr15_lopt34_10_0319  0.026077  0.027266  0.035737
20230814175110_satd_lexpr15_lopt34_10_0269  0.025901  0.027344  0.033700
20230814175115_satd_lexpr15_lopt34_10_0852  0.035336  0.036327  0.041362
20230815095228_satd_lexpr10_lopt34_10_0762  0.02167

In [50]:
finaldf.describe()

,y1,y2,y5
count,64.000000,64.000000,64.000000
mean,0.023043,0.024733,0.027598
std,0.006447,0.006554,0.008456
min,0.010208,0.011457,0.011241
25%,0.018774,0.020391,0.021407
50%,0.023624,0.025460,0.028360
75%,0.026870,0.029210,0.033970
max,0.036442,0.038170,0.044213
